## Hyperparameter Transfer

In the following example, we aim to find a single configuration 
from which we hope to achieve optimal performance on future datasets.

In this case, we are interested in a solution across `instances`. 
We therefore do **not** set the instance on construction.

In [49]:
from yahpo_gym import * 
import random
import math
import pandas as pd
# We do this on the 'lcbench' dataset
b = BenchmarkSet("lcbench")

In order to establish this, we define the `FindDefault` class, taking as input the target variable and the name of the instance column.

The method `find_default` then performs the search for a good default value:

It iteratively evaluates random configurations and keeps track of the configuration with optimal `mean` across training instances.

Note, that this example does not make use of multi-fidelity evaluations.

In [53]:
class FindDefault:
    """
    Find a default.
    """
    def __init__(self, target:str, instance_column:str):
        self.target = target
        self.instance_name = instance_column
        
        # First we split  the instances (= datasets) into train and  test instances
        self.train_ins = random.sample(b.instances, math.ceil(len(b.instances)*0.75))
        self.test_ins = [x for x in b.instances if x not in train_ins]
    
    def find_default(self, minimize, n_trials:int = 100):
        """
            Sequentially evaluate n_trials random configurations across all training instances
            in a given task only keeping the best.
        """
        best_val = float("inf")
        for i in range(n_trials):
            xs =  b.get_opt_space().sample_configuration(1).get_dictionary()
            val = self.eval_objfun_mean(xs, self.train_ins, self.instance_name)
            if not minimize:
                val = -val
            if val < best_val:
                best_val = val
                best_xs = xs
        
        best_xs.pop(self.instance_name, None)
        if not minimize:
            best_val = -best_val
        return best_xs, best_val
    
    def eval_default(self, xs):
        """
            Evaluate a default on test_instances
        """
        return self.eval_objfun_mean(xs, self.test_ins, self.instance_name)
        
    
    def eval_objfun_mean(self, xs, instances, ins_name):
        """
        Compute the mean of xs evaluated across all instances.
        """
        xb = self._batchify(xs, instances, ins_name)
        return pd.DataFrame.from_dict(b.objective_function(xb))[self.target].apply('mean')
        
    def _batchify(self, xs, instances, ins_name):
        """
        Turn a single configuration (Dictionary) into a list of configurations with different instances.
        """
        res = []
        for idx in instances:
            xc = xs.copy()
            xc.update({ins_name: idx})
            res += [xc]
        return res


In [54]:
fd = FindDefault('val_accuracy', 'OpenML_task_id')
xs, val = fd.find_default(minimize = False)
print(f"Configuration: \n{xs}\n")
print(f"With performance (train): {val}")

Configuration: 
{'batch_size': 216, 'epoch': 39, 'learning_rate': 0.015402621054460738, 'max_dropout': 0.22171831794360852, 'max_units': 402, 'momentum': 0.6501534298988096, 'num_layers': 2, 'weight_decay': 0.00223130033029889}

With performance (train): 75.36917114257812


We can now evaluate the performance of the found configuration `xs`.

In [55]:
val = fd.eval_default(xs)
print(f"With performance (test): {val}")

With performance (test): 71.50530242919922
